### Importing Disaster dataset into DisasterEvent and Location Ontology classes

In [5]:
import pandas as pd

In [3]:
%pip install rdflib

   ---------------------------------------- 0.0/564.9 kB ? eta -:--:--
   ------------------ --------------------- 262.1/564.9 kB ? eta -:--:--
   ---------------------------------------- 564.9/564.9 kB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD

In [3]:
# Load the ontology
g = Graph()
g.parse("FakeNewsOntology.rdf", format="xml")  # RDF/XML format

# Define namespace (Ensure it matches your ontology)
EX = Namespace("http://www.semanticweb.org/zakar/ontologies/2025/1/FakeNewsOntology#")
g.bind("ex", EX)

# Print the number of triples loaded
print(f"Ontology loaded with {len(g)} triples.")


Ontology loaded with 61 triples.


In [8]:
df = pd.read_csv("disaster_dataset.csv")
df

,DisNo.,Disaster Group,Disaster Subgroup,Disaster Subtype,Country,Subregion,Region,Start Year
0,1999-9388-DJI,Natural,Climatological,Drought,Djibouti,Sub-Saharan Africa,Africa,2001
1,1999-9388-SDN,Natural,Climatological,Drought,Sudan,Northern Africa,Africa,2000
2,1999-9388-SOM,Natural,Climatological,Drought,Somalia,Sub-Saharan Africa,Africa,2000
3,2000-0001-AGO,Technological,Transport,Road,Angola,Sub-Saharan Africa,Africa,2000
4,2000-0002-AGO,Natural,Hydrological,Riverine flood,Angola,Sub-Saharan Africa,Africa,2000
...,...,...,...,...,...,...,...,...
16182,2025-0101-USA,Technological,Transport,Air,United States of America,Northern America,Americas,2025
16183,2025-0102-NGA,Technological,Miscellaneous accident,Fire (Miscellaneous),Nigeria,Sub-Saharan Africa,Africa,2025
16184,2025-0103-KEN,Technological,Industrial accident,Collapse (Industrial),Kenya,Sub-Saharan Africa,Africa,2025
16185,2025-0104-USA,Technological,Transport,Air,United States of America,Northern America,Americas,2025


In [9]:
# Iterate through dataframe and map to ontology
for index, row in df.iterrows():
    # Create DisasterEvent instance
    event_id = URIRef(EX + "Disaster_" + str(row["DisNo."]))  # Unique URI
    g.add((event_id, RDF.type, EX.DisasterEvent))
    g.add((event_id, EX.disasterID, Literal(row["DisNo."], datatype=XSD.string)))
    g.add((event_id, EX.disasterGroup, Literal(row["Disaster Group"], datatype=XSD.string)))
    g.add((event_id, EX.disasterSubgroup, Literal(row["Disaster Subgroup"], datatype=XSD.string)))
    g.add((event_id, EX.disasterType, Literal(row["Disaster Subtype"], datatype=XSD.string)))
    g.add((event_id, EX.year, Literal(row["Start Year"], datatype=XSD.integer)))

    # Create Location instance
    location = URIRef(EX + "Location_" + str(row["Country"]).replace(" ", "_"))
    g.add((location, RDF.type, EX.Location))
    g.add((location, EX.country, Literal(row["Country"], datatype=XSD.string)))
    g.add((location, EX.region, Literal(row["Region"], datatype=XSD.string)))
    g.add((location, EX.subregion, Literal(row["Subregion"], datatype=XSD.string)))

    # Link DisasterEvent to Location
    g.add((event_id, EX.hasLocation, location))

# Save the updated ontology
g.serialize("Updated_FakeNewsOntology.rdf", format="xml")

print("Ontology successfully updated with disaster event data!")


Ontology successfully updated with disaster event data!
